In [144]:
import json
import requests
import numpy as np
import pandas as pd 
import regex as re

In [139]:

site_url = 'https://stlouis.myfeelstate.com/graphql?operationName=FilteredProducts&variables={"includeEnterpriseSpecials":false,"includeCannabinoids":true,"productsFilter":{"dispensaryId":"600f5a29d692585e7ffd25e4","pricingType":"med","strainTypes":[],"subcategories":[],"Status":"Active","types":[],"useCache":false,"sortDirection":1,"sortBy":null,"isDefaultSort":true,"bypassOnlineThresholds":false,"isKioskMenu":false,"removeProductsBelowOptionThresholds":true},"page":0,"perPage":500}&extensions={"persistedQuery":{"version":1,"sha256Hash":"aed681974c9093e0cc3099ef18a8d37e17e4916a9f7743dd372e0be6b13c8f10"}}'

r= requests.get(site_url)
data = r.json()


percentage_types = {'Flower':[],'Pre-Rolls':[],'Vaporizers':[],'Concentrate':[]}
dosage_types= {'Tincture':[],'Topicals':[], 'Edible':[],'CBD':[]}
accessory_type= {'Accessories':[],'Apparel':[]}

house_brands=['Robust','Lush Labs','Packs']
house_products=[]

product_data= data['data']['filteredProducts']['products']

for product in product_data:
    
    if product['type'] in percentage_types.keys():
        THC_content = str(product['THCContent']['range'][-1]).strip('[]'),+' '+ product['THCContent']['unit']
        CBD_content =  str(product['CBDContent']['range'][-1]).strip('[]'),' '+product['CBDContent']['unit']
        item= [product['type'], product['brandName'], product['Name'], product['THCContent']['range'],product['THCContent']['unit'], product['CBDContent']['range'],product['CBDContent']['unit'], product['Prices']]
        percentage_types[f'{product["type"]}'].append(item)
    
    elif product['type'] in dosage_types.keys():
        weight_value= str(product['measurements']['netWeight']['values']).strip('[]')
        weight_measurement= product['measurements']['netWeight']['unit']
        weight= ''.join(weight_value+' '+ weight_measurement)
        item=[product['type'], product['brandName'], product['Name'], weight, product['Prices']]
        dosage_types[f'{product["type"]}'].append(item)
    
    else:
        item=[product['type'], product['brandName'], product['Name'], product['Prices']]
        accessory_type[f'{product["type"]}'].append((product['Name'],product['Prices']))
        

TypeError: bad operand type for unary +: 'str'

In [133]:
for i in percentage_types.keys():

    for product in percentage_types[i]:
        print(product)

['Flower', "Mama J's", 'Sunset MAC - 3.5g', [23.44], 'PERCENTAGE', None, 'PERCENTAGE', [40]]
['Flower', 'Robust', 'Apple Cinnamon Crumble - 3.5g', [3.66], 'PERCENTAGE', [9.39], 'PERCENTAGE', [20]]
['Flower', 'Robust', 'Apple Fritter - 3.5g', [13.74], 'PERCENTAGE', [0.06], 'PERCENTAGE', [25]]
['Flower', 'Vibe Cannabis', 'Bakers Dozen - 3.5g', [24.02], 'PERCENTAGE', None, 'PERCENTAGE', [32]]
['Flower', 'Beach', 'Beach - Clementine -7g', [20.13], 'PERCENTAGE', [0.04], 'PERCENTAGE', [50]]
['Flower', 'Beach', 'Beach - Cookies & Cream (Uncut) - 7g', [22.15], 'PERCENTAGE', [0.06], 'PERCENTAGE', [40]]
['Flower', 'Robust', 'Biscotti Gas - 3.5g', [18.23], 'PERCENTAGE', [0.07], 'PERCENTAGE', [34]]
['Flower', 'Robust', 'Black Hole Sun - 3.5g', [17.57], 'PERCENTAGE', [0.04], 'PERCENTAGE', [28]]
['Flower', 'Cookies', 'Blanco - 3.5g', [27.22], 'PERCENTAGE', [0.03], 'PERCENTAGE', [50]]
['Flower', 'Flora Farms', 'Blue Dream - 3.5g', [13.69], 'PERCENTAGE', None, 'PERCENTAGE', [20]]
['Flower', 'Robust', 

In [138]:
for i in dosage_types.keys():

    for product in dosage_types[i]:
        print(product)

['Tincture', 'Care By Design', '2:1 CBD Drops | 15ml', '150 MILLIGRAMS', [38]]
['Tincture', 'Care By Design', '4:1 CBD Drops | 15ml', '750 MILLIGRAMS', [34]]
['Tincture', 'Vivid', 'Goodnight Grape 1:1 CBN Tincture', '100 MILLIGRAMS', [35]]
['Tincture', 'Lush Labs', 'Lush Labs | Live Rosin Full Spectrum Tincture | 400mg', '300 MILLIGRAMS', [50]]
['Tincture', 'Vivid', 'Morning Mint 1:1 THC:CBG Tincture', '100 MILLIGRAMS', [35]]
['Topicals', 'Care By Design', 'Care By Design - CBD Pain Cream -1:1 CBD:THC - 1.5oz', '100 MILLIGRAMS', [48]]
['Topicals', 'Care By Design', 'Care By Design | CBD Pain Cream | 1:1 CBD:THC', '100 MILLIGRAMS', [41]]
['Topicals', 'Heartland Labs', 'Heartland - FECO Suppositories - 25mg THC/ 10mg CBG/ 75mg CBD', '25 MILLIGRAMS', [40]]
['Topicals', 'Heartland Labs', 'Heartland Holistics FECO Transdermal Patch', '1750 MILLIGRAMS', [16]]
['Topicals', 'Heartland Labs', 'Heartland Holistics High Dose Transdermal Patch', '1400 MILLIGRAMS', [17]]
['Topicals', 'Heartland Lab

In [135]:
for i in accessory_type.keys():

    for product in accessory_type[i]:
        print(product)

('Jefe - Hempen Roach Clip', [30])
('4" Pulsar Isopropyl Cleaning Station - Assorted Colors', [40])
('4" Rising Sun Dugout - One Hitter Included', [10])
('4" Rustic Dugout - One Hitter Included', [10])
('4" Wood Dugout - One Hitter Included', [10])
('6" Pyramid Puck Bong/Rig - Assorted Colors', [25])
('AiroPro Vaporizer | Slate Gray', [30])
('AiroSport - Battery - Pink', [20])
('Battery - 510 Thread - Black', [10])
('Battery - 510 Thread - Silver', [10])
('Blazy Susan -  MINI Pink 53mm - 50 Ct', [20])
('Blazy Susan -  Pink 98mm Cones - 50 Ct', [20])
('Blazy Susan - Pink 1-1/4 Deluxe Rolling Kits', [4])
('EYCE - Proteck Steam Roller with LED - Assorted Colors', [40])
('EYCE - Shorty - Assorted Colors', [10])
('EYCE - Silicone Pipe Replacement Glass', [5])
('Eyce - Spoon', [17])
('Feel State - Lanyard', [10])
('Feel State - Lighter', [3])
('Feel State - Retractable Clip', [10])
('Feel State Grinder - Green (2 Tier)', [6])
('Feel State Grinder - Green (3 Tier)', [10])
('Feel State Grinder

In [165]:
for product in product_data:
    
    if product['type'] in percentage_types.keys():
        brand = product['brandName']
        name = product['Name']
        name= re.sub('[-|]', '', name)
        name = name.replace(brand, '')
        weight = re.search('g$', name)
        print(name.strip(), weight)

        if product['CBDContent']['range'] == None:
            pass

            THC_content = str(max(product['THCContent']['range'])).strip('[]')+' '+ product['THCContent']['unit']
            #CBD_content =  str(max(product['CBDContent']['range'])).strip('[]')+' '+product['CBDContent']['unit']
            item= [product['type'], product['brandName'], product['Name'], product['THCContent']['range'],product['THCContent']['unit'], product['CBDContent']['range'],product['CBDContent']['unit'], product['Prices']]
            percentage_types[f'{product["type"]}'].append(item)
    
    elif product['type'] in dosage_types.keys():
        weight_value= str(product['measurements']['netWeight']['values']).strip('[]')
        weight_measurement= product['measurements']['netWeight']['unit']
        weight= ''.join(weight_value+' '+ weight_measurement)
        item=[product['type'], product['brandName'], product['Name'], weight, product['Prices']]
        dosage_types[f'{product["type"]}'].append(item)
    
    else:
        item=[product['type'], product['brandName'], product['Name'], product['Prices']]
        accessory_type[f'{product["type"]}'].append((product['Name'],product['Prices']))

Sunset MAC  3.5g <regex.Match object; span=(15, 16), match='g'>
Seasonal Grapefruit Sour Cart None
Alien Rock Candy Badder  1g <regex.Match object; span=(26, 27), match='g'>
Animal Cookie Crisp PreRoll  1g <regex.Match object; span=(30, 31), match='g'>
Animal Mintz Disposable Cart  0.3g <regex.Match object; span=(33, 34), match='g'>
Appal Ape Wax  1g <regex.Match object; span=(16, 17), match='g'>
Apple Cinnamon Crumble  3.5g <regex.Match object; span=(27, 28), match='g'>
Apple Fritter  3.5g <regex.Match object; span=(18, 19), match='g'>
AppleNana Smoothie PreRoll  1g <regex.Match object; span=(29, 30), match='g'>
Apples & Bananas Blunt  1g <regex.Match object; span=(25, 26), match='g'>
Apples and Bandanas Blunt  0.5g <regex.Match object; span=(30, 31), match='g'>
Apricot Stomper #6 Live Resin Cartridge  0.5g <regex.Match object; span=(44, 45), match='g'>
Jurassic Kush Jam None
Atlanta Beach Pie Live Sugar  1g <regex.Match object; span=(31, 32), match='g'>
Backwood Tropics Live Resin Sa

In [150]:
for product in product_data:
    
    if product['type'] in percentage_types.keys():
        brand = product['type']
        name = product['Name']
        name= re.sub(f'[-,|,{brand}]', '', name)
        print(name)

Sunst MAC  3.5g
A  Snl Gfut Su Ct
Ali Rk dy Bdd  1g


error: bad character range at position 10

In [131]:
for i in dosage_types.keys():
    print(i)
    print(len(dosage_types[i]))

for i in percentage_types.keys():
    print(i)
    print(len(percentage_types[i]))


Tincture
5
Topicals
8
Edible
77
CBD
9
Flower
53
Pre-Rolls
27
Vaporizers
65
Concentrate
46


## PERCENTAGE CODE BLOCK